In [1]:
from queryAnalyzer import getPlayersInQuery, getLeaguesInQuery, getTeams, getSeasons, getStages
import sqlAnswers

In [2]:
import sqlite3
sqlite_file = 'database.sqlite'

query1 = "Thomas Mueller ist der beste Spieler der Welt!"
query1 = "Müller Thomas ist der beste Spieler der Welt, nicht Aaron Doran!"
query2 = "Bundesliga und Jupiler League sind supi!"
query2 = "Die deutsche Liga ist supi!"


In [3]:
#make query insensitive for umlaute
def preProcessQuery(query):
    umlaut_a = 'ä'.encode()
    umlaut_A = 'Ä'.encode()
    umlaut_o = 'ö'.encode()
    umlaut_O = 'Ö'.encode()
    umlaut_u = 'ü'.encode()
    umlaut_U = 'Ü'.encode()
    
    query = query.encode()
    
    query = query.replace(umlaut_a, b'ae')
    query = query.replace(umlaut_A, b'Ae')
    query = query.replace(umlaut_o, b'oe')
    query = query.replace(umlaut_O, b'Oe')
    query = query.replace(umlaut_u, b'ue')
    query = query.replace(umlaut_U, b'Ue')
    
    query = query.decode('utf-8')
    
    if(query[-1:] == '?' or query[-1:] == '!' or query[-1:] == '.' ):
        return query[:-1]
    return query

Query wird verarbeitet:
    
- Liefert eine playerID eines Spielers zurück
- Keyword "wiegt" (oder auch "schwer") wird gefunden

-> dieses keyword hat nen pointer auf eine SQL Query mit Platzhaltern, in welches die ID des Spielers von dem gesetzt wird


'SELECT weight FROM player WHERE id = playerID'


flair tool 
was passiert wenn der Input da rein kommt

Queries mit "Tags"

Spieler
Ligen
Verein (evtl abkürzung)
Saison / Jahr





In [4]:
from flair.embeddings import WordEmbeddings
from flair.data import Sentence
german_embedding = WordEmbeddings('de-crawl')
 
import numpy
import json

In [5]:
def getVectorOfAnswer(index):
    with open('trainData.json', 'r') as myfile:
        data=myfile.read()
    obj = json.loads(data)
    embeddings = 0
    for sentence in obj[str(index)]:
        
        sentence = Sentence(preProcessQuery(sentence))
        german_embedding.embed(sentence)
        
        embed = 0
        for token in sentence:
            #print(token)
            embed += token.embedding.numpy()
        #evtl embed durch die anzahl an Wörtern darin teilen ? beim Finalen auch?
        embeddings += embed
    return embeddings
#print(getVectorOfAnswer(0))

In [6]:
import numpy as np
def getVectorOfQuery(query):
    sentence = Sentence(query)
    german_embedding.embed(sentence)
    
    embed = 0
    for token in sentence:
        #print(token)
        embed += token.embedding.numpy()
    return embed


def unit_vector(vector):
    return vector / np.linalg.norm(vector)

def angle_between(v1, v2):
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))



In [12]:

query = preProcessQuery("Wann hat Manuel Neuer Geburtstag?")
queryV = getVectorOfQuery("Wann hat SPIELER Geburtstag") # erwarter preprocessed Query
season = []
queryRefined = [getPlayersInQuery(query), getLeaguesInQuery(query),getTeams(query),season]

answers = sqlAnswers.getAnswers()


queryToCompare = [len(queryRefined[0]),len(queryRefined[1]),len(queryRefined[2]),len(queryRefined[3])]
possibleAnswers = [] 
#contains index with possible ansers which could match the query depending
#on number of player occurances etc

for x in range(0,len(answers)): #check for every "answer", if [1,0,0,0] matches [1,0,0,0] etc
    sqlMatches = True;
    for i in range(0, len(queryToCompare)): #interate through the whole [0,1,2,3] array and check 
        if(queryToCompare[i] != answers[x][i]):  # if something doesnt match, throw it out
            sqlMatches = False
    if(sqlMatches):
        possibleAnswers.append(x)
        

print("In Frage kommende Anworten von der Anzahl der gefundenen Namen etc her: ")       
print(possibleAnswers)


angles = []
for answerIndex in possibleAnswers:
    angles.append(angle_between(getVectorOfAnswer(answerIndex),queryV))

print(angles)
print("Beste gefundene Antwort anhand der Cosinus Ähnlichkeit: " + str(angles.index(min(angles))))

print(answers[angles.index(min(angles))][4](queryRefined))

In Frage kommende Anworten von der Anzahl der gefundenen Namen etc her: 
[0, 1, 2, 3]
[0.3635899, 0.66552806, 0.73512274, 0.7025114]
Beste gefundene Antwort anhand der Cosinus Ähnlichkeit: 0
Der Geburtstag ist am 27.03.1986.
